In [ ]:
from outlines import models, generate, samplers
from torch import bfloat16

checkpoint = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
model = models.transformers(checkpoint, device='cuda', model_kwargs={"torch_dtype":bfloat16})
sampler = samplers.MultinomialSampler(temperature=0.5)

prompt = """You are an AI writing assistant. The bank is behind the fire station.
Question: Where is the bank?
Reply:
"""

In [4]:
generator = generate.text(model, sampler)
answer = generator(prompt, max_tokens=20, stop_at=['.', '!'])
print(answer)

The bank is located behind the fire station.
